Week 3 Submission : Segmenting and Clustering Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

To create the above dataframe:

1.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

2.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 
 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will    notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
 
3.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [1]:
## Import Libraries
import pandas as pd
import requests #  library to handle requests
from bs4 import BeautifulSoup
print("Libraries imported")

Libraries imported


In [2]:
#!conda install --yes beautifulsoup4

In [3]:
## Download the data from URL and convert into xml format

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
Source_Data=BeautifulSoup(source,'lxml')

#Source_Data

In [4]:
## Create a Data frame
column_names=['PostCode','Borough','Neighborhood']
Postal_Codes=pd.DataFrame(columns=column_names)

In [5]:
# loop through to find postcode, borough, neighborhood 

content = Source_Data.find('div', class_='mw-parser-output')

table = content.table.tbody
#print(table)
v_postcode = 0
v_borough = 0
v_neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            v_postcode = td.text
            i = i + 1
        elif i == 1:
            v_borough = td.text
            i = i + 1
        elif i == 2: 
            v_neighborhood = td.text.strip('\n').replace(']','')        
    Postal_Codes = Postal_Codes.append({'PostCode': v_postcode,'Borough': v_borough,'Neighborhood': v_neighborhood},ignore_index=True)


In [6]:
print("Orignial Count of Dataframe:",Postal_Codes.shape)

Orignial Count of Dataframe: (289, 3)


In [7]:
## Cleaning the dataframe 
## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
Postal_Codes=Postal_Codes[Postal_Codes.Borough!=0]
Postal_Codes=Postal_Codes[Postal_Codes.Borough!='Not assigned']
## reindexing the dataframe
Postal_Codes.reset_index(drop=True,inplace=True)
print("Revised Count of DataFrame that have assigned Borough:",Postal_Codes.shape)

Revised Count of DataFrame that have assigned Borough: (211, 3)


In [8]:
## Data Modification 

## 1. Updating the Neighborhood name with Borogh name if its Not Assigned

i = 0
for i in range(0,Postal_Codes.shape[0]):
    if Postal_Codes.iloc[i][2] == 'Not assigned':
        Postal_Codes.iloc[i][2] = Postal_Codes.iloc[i][1]
        i = i+1

        
## 2. More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma .

Postal_Codes = Postal_Codes.groupby(['PostCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
Postal_Codes.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
print("Final Count of DataFrame:",Postal_Codes.shape)

Final Count of DataFrame: (103, 3)


In [26]:
## Reading the file provided by Coursera'
geo_df=pd.read_csv("http://cocl.us/Geospatial_data")
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
Postal_Codes.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
## Renaming the column name to match the column names in the Postal Codes Data Frame 

geo_df.rename(columns={"Postal Code":"PostCode"},inplace=True)

## Merging Data Frames on column PostCode

merged_df=pd.merge(Postal_Codes,geo_df,on='PostCode')
merged_df.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
